# Neptune + PyTorch

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/scale-examples/blob/lr%2Fpytorch_example/integrations-and-supported-tools/pytorch/notebooks/pytorch_text_model_debugging.ipynb"> 
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/> 
</a>

## Introduction
Global metrics such as loss or accuracy provide a high-level performance snapshot and ensure training is on course.

However, for large or foundation models, monitoring layer-wise metrics—such as gradients and activations—delivers critical insights into how each layer learns. This level of detail helps identify issues and fine-tune individual layers for better overall performance.
The main challenge is the volume of data generated by layer-wise logging.

Fortunately, Neptune is built for hyperscale tracking. It enables you to capture, organize, and analyze metrics from every layer without disrupting the training process. No matter how large is your model.

This guide will show you how to:
- Initialize the **Neptune Run** object and log configuration parameters
- Create a **reusable class** to hook and log layer-wise metrics (`TorchWatcher`)
- Log **aggregated metrics** such as loss and accuracy
- Log **layer-wise metrics** to debug model training such as:

| **Metric**                        | **Demonstrated** | **What it shows**                                                                                             | **How to capture**                                             |
|-----------------------------------|--------------------------------------|--------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------|
| **Activations**                   | Yes                                  | Dead or exploding activations can indicate issues with training stability. | `TorchWatcher`            |
| **Gradients**                     | Yes                                  | Essential for diagnosing vanishing or exploding gradients. Small gradients may indicate vanishing gradients, while large ones can signal instability. | `TorchWatcher`        |
| **Parameters**            | Yes                                  | Tracks how the model’s parameters evolve during training. Large or small weights may indicate the need for better regularization or adjustments in learning rate. | Extract directly from the model’s parameters.                  |
| **Loss**               | No                                   | Identifies which parts of the network contribute more to the overall loss, aiding debugging and optimization. | Monitor outputs from each layer and compare with the target.   |
| **Learning rate**       | No                                   | Helpful if using techniques like Layer-wise Learning Rate Decay (L2LRD). Tracking this can provide insight into the layer-specific learning rate. | Manually track based on optimizer settings.                    |
| **Output norms**            | No                                   | The L2-norm of layer outputs can highlight issues like gradient explosion or vanishing gradients. | Compute the L2-norm for each layer’s output.                   |

Use this notebook as a code recipe. Add your own code and adapt the sections to your own model training needs.

## Before you start

  1. Create a Neptune Scale account. [Register &rarr;](https://neptune.ai/early-access)
  2. Create a Neptune project for tracking metadata. For instructions, see [Projects](https://docs-beta.neptune.ai/projects/) in the Neptune Scale docs.
  3. Install and configure Neptune Scale for logging metadata. For instructions, see [Get started](https://docs-beta.neptune.ai/setup) in the Neptune Scale docs.

### Set environment variables
By setting your project name and API token as environment variables, you can use them throughout this notebook.

Uncomment the code block below and replace placeholder values with your own credentials:

In [ ]:
# Set Neptune credentials as environment variables
# %env NEPTUNE_API_TOKEN = "your_api_token"
# %env NEPTUNE_PROJECT = "your_workspace_name_here/your_project_name"

### Install dependencies and import libraries

In [ ]:
# Install dependencies
! pip install -qU neptune_scale torch datasets

In [1]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from collections import Counter
from datasets import load_dataset

from neptune_scale import Run
import os

c:\Users\leo.breedt\miniconda3\envs\neptune_scale_py_312_base\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initialize parameters

In [2]:
# Initialize model, loss function, and optimizer

params = {
    "optimizer": "Adam",
    "batch_size": 8,
    "learning_rate": 0.01,
    "epochs": 5,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "input_features": 256,
    "embed_size": 1000,
    "hidden_size": 256,  # hidden size for the LSTM
    "dropout_prob": 0.3,
    "num_lstm_layers": 3,
}

## Download or use next token prediction dataset
This example uses the dataset from [HuggingFace](https://huggingface.co/datasets/Na0s/Next_Token_Prediction_dataset) (HF).

You can increase the size of the dataset to test the logging capabilities of Neptune. Note that increasing the size will increase the time needed for the dataset to download. The current setup only downloads the first parquet file from the Hugging Face public dataset.

The validation dataset is also reduced to decrease the training loop execution time. To increase the validation size, change the `test_size` key-value pair in the `train_test_split()` method from HuggingFace.

In [3]:
# For the example, download a random subset of 10% of the original dataset
base_url = "https://huggingface.co/datasets/Na0s/Next_Token_Prediction_dataset/resolve/main/data/"
data_files = {
    "train": base_url
    + "train-00001-of-00067.parquet",  # download only the first 10 files from the HF dataset
    "validation": base_url + "validation-00000-of-00001.parquet",
}  # download the complete validation dataset

data_subset = load_dataset("parquet", data_files=data_files, num_proc=4)
# validation_subset = load_dataset("parquet", data_files = {"validation": base_url + "validation-00000-of-00001.parquet"}, num_proc=4, split=["validation[:5%]"])
validation_subset = data_subset.get("validation").train_test_split(test_size=0.1)
print(
    f"Training samples: {data_subset['train'].num_rows} \nValidation samples: {validation_subset['test'].num_rows}"
)

Training samples: 81926 
Validation samples: 935


## Create `DataLoader` objects
To execute the models with PyTorch, convert the training and validation datasets to tensors. Then, set up `DataLoader` for easier batching in the training loop.

The model architecture requires the vocabulary size as an input and this is why we calculate the max token from the dataset.

In [4]:
train_subset = data_subset["train"].with_format(
    type="torch", columns=["text", "input_ids", "labels"]
)  # HF provides methods to convert data types to tensors
validation_subset = validation_subset["test"].with_format(
    type="torch", columns=["text", "input_ids", "labels"]
)  # HF provides methods to convert data types to tensors

train_dataloader = DataLoader(train_subset, batch_size=params["batch_size"], shuffle=True)
val_dataloader = DataLoader(validation_subset, batch_size=params["batch_size"], shuffle=True)

# Determine the vocab size of the dataset
# Flatten the list of tokenized sentences into one long list of token IDs
vocab_size = (
    max([token for sentence in data_subset["train"]["input_ids"] for token in sentence]) + 1
)
params["vocab_size"] = vocab_size
print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 128257


### Define PyTorch model architecture and helpers
Define a simple LLM model architecture using PyTorch. Since this is a text-based example, we use an embedding layer, a LSTM layer, and a fully connected layer.

You can adjust this architecture to your needs and increase its size when testing the workflow:
- To increase the size of the LSTM layers, change the `num_layers` parameter in the parameters dictionary.
- To increase the number of fully connected layers, update the mode architecture itself.

In [5]:
# Define the simple LLM model with LSTM
class SimpleLLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(SimpleLLM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)  # LSTM returns output and hidden/cell state tuple
        out = self.fc1(lstm_out)  # Use the last output from the LSTM
        return out


# Function to evaluate the model after each epoch/step
def evaluate(model, val_dataloader, criterion, device, vocab_size):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    with torch.no_grad():  # Disable gradient calculation for validation
        for batch in val_dataloader:
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            # Forward pass for validation
            logits = model(input_ids)  # Shape: (batch_size, seq_len, vocab_size)

            # Calculate the loss
            loss = criterion(logits.view(-1, vocab_size), labels.view(-1))
            total_loss += loss.item()

    avg_val_loss = total_loss / len(val_dataloader)
    return avg_val_loss

### Setup tracking class

This section intializes the `TorchWatcher` class:
- It accepts a PyTorch model object as an input. 
- It allows you to capture the **parameters**, **activations** and **gradients** from each layer.
- Specify which tensor statistics to capture.

See a pseudo implementation:

```python
from TorchWatcher import TorchWatcher

model = YourModel()
watcher = TorchWatcher(model, run)  # Uses default mean() statistic

# Training loop
for epoch in range(3):
    # Forward pass
    output = model(x_batch)
    loss = criterion(output, y_batch)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Track metrics after the forward and backward passes
    watcher.watch(step=epoch)
```

In [6]:
from TorchWatcher import TorchWatcher

## Set up model training

### Initialize Neptune run object and log hyperparameters

In [15]:
from neptune_scale import Run
from uuid import uuid4

custom_run_id = f"pytorch-text-{uuid4()}"  # Create your own custom run_id
experiment_name = "pytorch-text"  # Create a run that is the head of an experiment. This will also be used for forking.

run = Run(
    run_id=custom_run_id,
    experiment_name=experiment_name,
)

run.log_configs(
    {
        "config/learning_rate": params["learning_rate"],
        "config/optimizer": params["optimizer"],
        "config/batch_size": params["batch_size"],
        "config/epochs": params["epochs"],
        "config/num_lstm_layers": params["num_lstm_layers"],
        "data/vocab_size": params["vocab_size"],
        "data/embed_size": params["embed_size"],
    }
)

run.add_tags(tags=[params["optimizer"]], group_tags=True)
run.add_tags(tags=["text", "LLM", "Simple"])

print(run.get_experiment_url())

https://scale.neptune.ai/leo/pytorch-tutorial/runs/details?runIdentificationKey=pytorch-text&type=experiment


### Execute model training loop

In the training loop we call the `watch()` method from the `TorchWatcher` package after the backward and forward passes to track our parameters, gradients and activations with a combination of tensor statistics. 

In [16]:
# Initialize model and optimizer
model = SimpleLLM(
    params["vocab_size"], params["embed_size"], params["hidden_size"], params["num_lstm_layers"]
)
optimizer = optim.Adam(model.parameters(), lr=params["learning_rate"])
criterion = nn.CrossEntropyLoss(
    ignore_index=-100
)  # Ignore the buffering index of -100 in the dataset

# Define watcher class
watcher = TorchWatcher(model=model, run=run, tensor_stats=["mean", "norm"], base_namespace="debug_metrics")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
step_counter = 0

# Training loop
for epoch in range(params["epochs"]):
    total_loss = 0
    for batch in train_dataloader:
        model.train()
        step_counter += 1

        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        # Forward pass
        logits = model(input_ids)

        # Compute the loss (ignore padding tokens by masking labels)
        loss = criterion(logits.view(-1, vocab_size), labels.view(-1))

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        print(f"Step {step_counter} / {len(train_dataloader)}, Loss: {loss.item()}")

        # Call watch() method in loop determing which layer-wise metrics to watch
        watcher.watch(step=step_counter, track_activations=False, track_parameters=False)

        if step_counter % 50 == 0:  # Log validation loss at every 50 steps
            val_loss = evaluate(model, val_dataloader, criterion, device, vocab_size)

            run.log_metrics(
                data={
                    "metrics/train/loss": loss.item(),
                    "metrics/validation/loss": val_loss,
                    "epoch/value": epoch,
                },
                step=step_counter,
            )
        else:  # Log training loss and debugging metrics for each step
            run.log_metrics(
                data={"metrics/train/loss": loss.item(), "epoch/value": epoch},
                step=step_counter,
            )

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_dataloader)}")

# Close run to ensure all operations are processed
run.close()

Step 1 / 10241, Loss: 11.765020370483398
Step 2 / 10241, Loss: 11.628341674804688
Step 3 / 10241, Loss: 9.686809539794922
Step 4 / 10241, Loss: 9.657011032104492
Step 5 / 10241, Loss: 9.74715518951416
Step 6 / 10241, Loss: 10.450525283813477
Step 7 / 10241, Loss: 10.163161277770996
Step 8 / 10241, Loss: 10.278069496154785
Step 9 / 10241, Loss: 10.855117797851562
Step 10 / 10241, Loss: 10.982325553894043
Step 11 / 10241, Loss: 10.332056999206543
Step 12 / 10241, Loss: 10.331168174743652
Step 13 / 10241, Loss: 10.291738510131836
Step 14 / 10241, Loss: 9.950544357299805


KeyboardInterrupt: 

## What's next?
While the model is training, you can start using the Neptune web app to browse your metrics and create custom analyses and visualizations:
1. To visualize the large number of metrics being logged in near real time, navigate to the **Charts** tab of the active run.
2. Filter the metrics using the [advanced regex searching capabilities](https://docs-beta.neptune.ai/charts#filtering-charts). For example, enter `.*gradient+.*fc\d` in the search bar. This query filters all metrics for the gradients of the fully connected layers. The more FC layers, the more charts will appear.
3. Export the filter to a [dashboard](https://docs-beta.neptune.ai/custom_dashboard). The saved dashboard will now only display these metrics during training.
4. Use the [dynamic metric selection](https://docs-beta.neptune.ai/chart_widget#dynamic-metric-selection) and update the chart widget to display all fully connected layers gradients in one chart. Again, use the `.*gradient+.*fc\d` query.
5. Create a [custom report](https://docs-beta.neptune.ai/reports) to outline the model training, global metrics, debugging metrics, and more.

See also [a generic example of the training result](https://scale.neptune.ai/o/examples/org/LLM-Pretraining/reports/9e6a2cad-77e7-42df-9d64-28f07d37e908).
